# AutoGen അടിസ്ഥാന സാമ്പിൾ

ഈ കോഡ് സാമ്പിളിൽ, നിങ്ങൾ [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework ഉപയോഗിച്ച് ഒരു അടിസ്ഥാന ഏജന്റ് സൃഷ്ടിക്കും.

വിവിധ ഏജന്റിക് പാറ്റേണുകൾ നടപ്പിലാക്കുമ്പോൾ നമുക്ക് പിന്നീട് ഉപയോഗിക്കേണ്ട ഘട്ടങ്ങൾ ഈ സാമ്പിൾ കാണിക്കുന്നതാണ് ഈ ഉദാഹരണത്തിന്റെ ലക്ഷ്യം.


## ആവശ്യമായ പൈതൺ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## ക്ലയന്റ് സൃഷ്ടിക്കുക

ഈ ഉദാഹരണത്തിൽ, LLM-ലേക്ക് ആക്സസ് നേടാൻ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ഉപയോഗിക്കും.

`model` എന്നത് `gpt-4o-mini` ആയി നിർവചിച്ചിരിക്കുന്നു. GitHub Models മാർക്കറ്റ്പ്ലേസിൽ ലഭ്യമായ മറ്റൊരു മോഡലിലേക്ക് മാറ്റി വ്യത്യസ്ത ഫലങ്ങൾ കാണാൻ ശ്രമിക്കുക.

ഒരു ദ്രുത പരിശോധനയായി, നാം ഒരു ലളിതമായ പ്രോംപ്റ്റ് പ്രവർത്തിപ്പിക്കും - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ഏജന്റ് നിർവചിക്കുന്നത്

നാം `client` സജ്ജമാക്കി അത് പ്രവർത്തിക്കുന്നുണ്ടെന്ന് സ്ഥിരീകരിച്ചതിനുശേഷം, ഒരു `AssistantAgent` സൃഷ്ടിക്കാം. ഓരോ ഏജന്റിനും താഴെ പറയുന്നവ നൽകാം:  
**name** - മൾട്ടി-ഏജന്റ് പ്രവാഹങ്ങളിൽ ഇത് പരാമർശിക്കാൻ ഉപകാരപ്രദമായ ഒരു ചുരുക്കനാമം.  
**model_client** - നിങ്ങൾ മുൻപത്തെ ഘട്ടത്തിൽ സൃഷ്ടിച്ച client.  
**tools** - ഏജന്റ് ഒരു ടാസ്ക് പൂർത്തിയാക്കാൻ ഉപയോഗിക്കാവുന്ന ഉപകരണങ്ങൾ.  
**system_message** - ടാസ്ക്, പെരുമാറ്റം, LLM-ന്റെ ശൈലി എന്നിവ നിർവചിക്കുന്ന metaprompt.  

LLM എങ്ങനെ പ്രതികരിക്കുന്നുവെന്ന് കാണാൻ system message മാറ്റാം. `tools`-നെ കുറിച്ച് നാം പാഠം #4-ൽ വിശദീകരിക്കും.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ഏജന്റ് പ്രവർത്തിപ്പിക്കുക

താഴെ നൽകിയിരിക്കുന്ന ഫംഗ്ഷൻ ഏജന്റിനെ പ്രവർത്തിപ്പിക്കും. പുതിയ സന്ദേശത്തോടെ ഏജന്റിന്റെ നില അപ്ഡേറ്റ് ചെയ്യാൻ ഞങ്ങൾ `on_message` മെത്തഡ് ഉപയോഗിക്കുന്നു.

ഈ സാഹചര്യത്തിൽ, ഉപയോക്താവിൽ നിന്ന് `"Plan me a great sunny vacation"` എന്ന പുതിയ സന്ദേശം ഉപയോഗിച്ച് നില അപ്ഡേറ്റ് ചെയ്യുന്നു.

സന്ദേശത്തിന്റെ ഉള്ളടക്കം മാറ്റിയാൽ LLM വ്യത്യസ്തമായി പ്രതികരിക്കുന്നതെങ്ങനെയെന്ന് കാണാൻ കഴിയും.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖയാണ് വിശ്വസനീയമായ ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
